I scrape the table from wiki page. 
check and found that the neighborhood is already combined by PostCode
check and found that there is no row have  (Borough!= Not assigned' and Neighborhood==Not assinged)
So the df is the final result

In [1]:
import pandas as pd

dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') 

In [2]:
df = dfs[0]
df.columns = ['PostalCode','Borough','Neighborhood']
df.head(10)

,PostalCode,Borough,Neighborhood
0,Postal Code,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"


In [3]:
df = df.drop(index=0) # delete one row
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
print(df[df.PostalCode=='M5A'])
print(df.PostalCode.count()==len(df))
# so it already satisfy

  PostalCode           Borough               Neighborhood
5        M5A  Downtown Toronto  Regent Park, Harbourfront
True


In [5]:
df_part = df[df.Neighborhood =='Not assigned']
df_part

df_part[df_part.Borough != 'Not assigned']
df_part
df_part = df_part[df_part.Borough != 'Not assigned']
df_part
#so there is no Borough!= Not assigned' and Neighborhood==Not assinged

,PostalCode,Borough,Neighborhood


In [17]:
df = df[df.Borough!='Not assigned']
df.shape

(103, 3)

In [25]:
df_location = pd.read_csv('Geospatial_Coordinates.csv')
df_location.rename(columns={"Postal Code": "PostalCode"},inplace=True)
df_location.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [32]:
df_combine = pd.merge(df, df_location, how='left', on='PostalCode')
df_combine.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [54]:
print(df_combine.Borough.unique())
print(len(df_combine.Borough.unique()))

['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga']
10


Create a map of New York with neighborhoods superimposed on top.

In [81]:
import folium # map rendering library
import numpy as np

# latitude = list(df_combine.Latitude)
# longitude=list(df_combine.Longitude)
# map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)


# create map of New York using latitude and longitude values
map_tt = folium.Map(location=[44, -79.9], zoom_start=1)
map_tt
# add markers to map
for lat, lng, borough, neighborhood in zip(df_combine['Latitude'], df_combine['Longitude'], df_combine['Borough'], df_combine['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [95]:
toronto_data = df_combine[(df_combine['Borough'] == 'Downtown Toronto') | (df_combine['Borough'] == 'East Toronto') | (df_combine['Borough'] == 'Central Toronto') | (df_combine['Borough'] == 'West Toronto')].reset_index(drop=True)
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [96]:
# create map of New York using latitude and longitude values
map_tt = folium.Map(location=[44, -79.9], zoom_start=1)
map_tt
# add markers to map
for lat, lng, borough, neighborhood in zip(df_combine['Latitude'], df_combine['Longitude'], df_combine['Borough'], df_combine['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [97]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

# manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)
manhattan_grouped_clustering = toronto_data[['Latitude','Longitude']]
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 3, 1, 1, 2, 1, 2], dtype=int32)

In [98]:
kmeans.labels_

array([1, 1, 1, 1, 3, 1, 1, 2, 1, 2, 1, 2, 3, 1, 2, 3, 1, 3, 0, 0, 0, 0,
       2, 0, 1, 2, 0, 1, 2, 0, 1, 0, 1, 1, 1, 1, 1, 1, 3], dtype=int32)